In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import tarfile
import os
import csv
#import shapefile
import geopandas as gpd
from zipfile import ZipFile

/home/nardo/.conda/envs/ic/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
pwd

'/home/nardo/extractData'

In [3]:
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

In [4]:
def descomp(path):
    zip_files = [f.path for f in os.scandir(path) if not f.is_dir()]
    for zip_file in zip_files:
        print(zip_file)
        with zipfile.ZipFile(zip_file, 'r') as z:
            z.extractall(path=path+'files')
#descomp('Dados_Radar_Jaraguarí/')

In [7]:
shp = "Localizacao_Remotas-FINAL.shp"
shape = gpd.read_file(filename=shp)

## WGS84
- Sistema de coordenada Projetado
- Sistema de coordenada Geodésico

In [8]:
shape.crs

{'init': 'epsg:4326'}

In [9]:
cellsize = 0.005825080633
ncols = 835
nrows = 779
lat_min = -22.542179540896
lat_max = lat_min + cellsize * nrows
lon_min = -56.903502116470
lon_max = lon_min + cellsize * ncols

In [10]:
lat2 = np.linspace(lat_max, lat_min, nrows, endpoint=True)
lat3 = np.around(lat2, decimals=4)
lon2 = np.linspace(lon_min, lon_max, ncols, endpoint=True)
lon3 = np.around(lon2, decimals=4)


# TODO: Conferir se näo esta invertido
shape['longitude'] = shape['geometry'].apply(lambda x: x.x)
shape['latitude'] = shape['geometry'].apply(lambda x: x.y)

shape['index_longitude'] = shape['longitude'].apply(
                                                lambda x: find_nearest(lon3, x))
shape['index_latitude'] = shape['latitude'].apply(
                                                lambda x: find_nearest(lat3, x))


In [11]:
shape.head()

,Name,Descriptio,geometry,longitude,latitude,index_longitude,index_latitude
0,MB SEG2,None,POINT Z (-54.6296 -20.44719999899997 0),-54.629600,-20.447200,390,419
1,MB LAG3,None,POINT Z (-54.67529999999999 -20.50189999899993 0),-54.675300,-20.501900,382,428
2,MB SEG3A,None,POINT Z (-54.63569401499996 -20.41419341199997 0),-54.635694,-20.414193,389,413
3,MB LAG2,None,POINT Z (-54.6867 -20.51749999899994 0),-54.686700,-20.517500,380,431
4,MB LAJ8,None,POINT Z (-54.5606 -20.47149999899994 0),-54.560600,-20.471500,402,423


In [12]:
points = shape.Name.unique()
points = list(map(lambda x: x.replace(" ", "_"), points))

In [13]:
shape.Name = points
shape = shape.set_index('Name')
shape.head()

,Descriptio,geometry,longitude,latitude,index_longitude,index_latitude
Name,,,,,,
MB_SEG2,None,POINT Z (-54.6296 -20.44719999899997 0),-54.629600,-20.447200,390,419
MB_LAG3,None,POINT Z (-54.67529999999999 -20.50189999899993 0),-54.675300,-20.501900,382,428
MB_SEG3A,None,POINT Z (-54.63569401499996 -20.41419341199997 0),-54.635694,-20.414193,389,413
MB_LAG2,None,POINT Z (-54.6867 -20.51749999899994 0),-54.686700,-20.517500,380,431
MB_LAJ8,None,POINT Z (-54.5606 -20.47149999899994 0),-54.560600,-20.471500,402,423


In [14]:
shape[shape.index == "MB_PRO1"]

,Descriptio,geometry,longitude,latitude,index_longitude,index_latitude
Name,,,,,,
MB_PRO1,None,POINT Z (-54.59739982999997 -20.46463848599995 0),-54.5974,-20.464638,395,422


In [17]:
path = 'Data_Radar_new/'
folders = [f.path for f in os.scandir(path) if f.is_dir()]
folders

['Data_Radar_new/2017-03',
 'Data_Radar_new/2017-10',
 'Data_Radar_new/2016-10',
 'Data_Radar_new/2017-05',
 'Data_Radar_new/2016-02',
 'Data_Radar_new/2018-01',
 'Data_Radar_new/2016-11',
 'Data_Radar_new/2017-07',
 'Data_Radar_new/2018-06',
 'Data_Radar_new/2018-04',
 'Data_Radar_new/2017-12',
 'Data_Radar_new/2017-01',
 'Data_Radar_new/2016-12',
 'Data_Radar_new/2016-01',
 'Data_Radar_new/2016-03',
 'Data_Radar_new/2018-05',
 'Data_Radar_new/2018-02',
 'Data_Radar_new/2017-06',
 'Data_Radar_new/2017-11',
 'Data_Radar_new/2016-04',
 'Data_Radar_new/2016-06',
 'Data_Radar_new/2017-02']

In [20]:
from tqdm import tqdm, tqdm_notebook
import logging


logging.basicConfig(filename='extractFiles2017.log',level=logging.DEBUG)


with open('Data_Radar_new/data.csv','w') as f_out:
    
    logging.info("Starting!\nCreating file %s",f_out)
    out_colnames = ['date']
    out_colnames += points
    
    writer = csv.DictWriter(f_out, fieldnames=out_colnames)
    writer.writeheader()
    path = 'Data_Radar_new/'
    folders = [f.path for f in os.scandir(path) if f.is_dir()]

    

    for folder in tqdm_notebook(folders,desc="Folders"):
        logging.info("Folder: %s", folder)

        zipfiles = [f.path for f in os.scandir(folder) if not f.is_dir()]
        
        
        for zipfile in tqdm_notebook(zipfiles, desc="ZipFiles", leave= False):
            logging.info("Zipfile: %s", zipfile)
            
            with ZipFile(zipfile,'r') as myzip:
                for file in tqdm_notebook(myzip.namelist(),desc="Files",leave= False):
                    logging.info("File: %s",file)
                    with myzip.open(file,'r') as data:      
                        new_point = {}
                        year   = file[:4]
                        month  = file[4:6]
                        day    = file[6:8]
                        hour   = file[8:10]
                        minute = file[10:12]
                        
                        #Dateformat %Y/%m/%d %H:%M
                        new_point['date'] = "{}/{}/{} {}:{}".format(year,month,day,hour,minute)
                        try:
                            array = np.loadtxt(data,skiprows=6)


                            if array.shape == (779,835):
                                array = np.power([10],array / 10)

                                for point in points:

                                    lat = shape.loc[point]['index_latitude']
                                    long = shape.loc[point]['index_longitude']
                                    lat_min = lat - 2
                                    lat_max = lat + 3 
                                    long_min = long - 2
                                    long_max = long + 3
                                    points_array = array[lat_min:lat_max,long_min:long_max]
                                    points_array = points_array.reshape(25,)
                                    points_string = np.array2string(points_array)
                                    
                                    for symbol in ['[',']']:
                                        points_string = points_string.replace(symbol,'')
                                    

                                    new_point[point] = points_string
                            else:
                                #nome dos arquivos para uma futura investigação
                                logging.warn("File wrong size: %s")

                            writer.writerow(new_point)
                        except ValueError:
                            logging.warn("File with strange data")
                        except:
                            logging.warn("SOME PROBLEM")
                        
                            

/home/nardo/.conda/envs/ic/lib/python3.6/site-packages/ipykernel_launcher.py:73: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
/home/nardo/.conda/envs/ic/lib/python3.6/site-packages/ipykernel_launcher.py:69: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead


/home/nardo/.conda/envs/ic/lib/python3.6/site-packages/ipykernel_launcher.py:75: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead


In [ ]:
df = pd.read_csv("Data_Radar_new/data.csv",
            parse_dates=['date'],
            index_col=["date"])

In [ ]:
df.head()